# Fruit Recognition with YOLOv8
Training and Testing Notebook for 9 Fruit Classes Detection
**Classes:** Apple, Banana, Cherry, Lemon, Orange, Peach, Pear, Strawberry, Watermelon

## 1. Import Required Libraries

In [ ]:
!pip install -q ultralytics opencv-python pillow numpy pandas matplotlib pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.3 MB/s eta 0:00:00


In [ ]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import yaml
import shutil
from ultralytics import YOLO
from google.colab import drive
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ All libraries imported successfully!


## 2. Mount Google Drive and Load Dataset

In [ ]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
print("✓ Google Drive mounted successfully!")

# Check if Fruits_data folder exists
fruits_data_path = '/content/drive/MyDrive/Fruits_data'
if os.path.exists(fruits_data_path):
    print(f"✓ Fruits_data folder found at: {fruits_data_path}")
    print(f"Contents: {os.listdir(fruits_data_path)}")
else:
    print(f"✗ Fruits_data folder not found at: {fruits_data_path}")
    print("Make sure you have uploaded the Fruits_data folder to Google Drive root")

Mounted at /content/drive
✓ Google Drive mounted successfully!
✓ Fruits_data folder found at: /content/drive/MyDrive/Fruits_data
Contents: ['test', 'train', 'valid']


## 3. Prepare Dataset Configuration

In [ ]:
# Define dataset paths
DATASET_ROOT = '/content/drive/MyDrive/Fruits_data'
WORK_DIR = '/content/fruit_detection'

# Create working directory
os.makedirs(WORK_DIR, exist_ok=True)

# Define 9 fruit classes
CLASSES = {
    0: 'apple',
    1: 'banana',
    2: 'cherry',
    3: 'lemon',
    4: 'orange',
    5: 'peach',
    6: 'pear',
    7: 'strawberry',
    8: 'watermelon'
}

print(f"Classes: {CLASSES}")

# Create data.yaml configuration for YOLOv8
data_yaml = {
    'path': DATASET_ROOT,
    'train': f'{DATASET_ROOT}/train/images',
    'val': f'{DATASET_ROOT}/valid/images',
    'test': f'{DATASET_ROOT}/test/images',
    'nc': 9,
    'names': list(CLASSES.values())
}

# Save YAML file
yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"✓ YAML config created at: {yaml_path}")
print(f"✓ Dataset path: {DATASET_ROOT}")
print(f"✓ Working directory: {WORK_DIR}")

Classes: {0: 'apple', 1: 'banana', 2: 'cherry', 3: 'lemon', 4: 'orange', 5: 'peach', 6: 'pear', 7: 'strawberry', 8: 'watermelon'}
✓ YAML config created at: /content/fruit_detection/data.yaml
✓ Dataset path: /content/drive/MyDrive/Fruits_data
✓ Working directory: /content/fruit_detection


## 4. Download and Initialize YOLOv8 Model

In [ ]:
# Load YOLOv8 model for transfer learning
# YOLOv8m is a good balance between speed and accuracy
# You can also use YOLOv8s (small), YOLOv8n (nano), YOLOv8l (large), YOLOv8x (xlarge)
model = YOLO('yolov8m.pt')

print("✓ YOLOv8m model loaded successfully!")
print(f"Model summary:")
model.info()

✓ YOLOv8m model loaded successfully!
Model summary:
YOLOv8m summary: 169 layers, 25,902,640 parameters, 0 gradients, 79.3 GFLOPs


(169, 25902640, 0, 79.3204224)

## 5. Train YOLOv8 Model

In [1]:
# Train YOLOv8 Model with Resumable Training & Auto-Save
import subprocess, sys, os, torch, yaml, shutil
from ultralytics import YOLO

# Install ultralytics if needed
try:
    from ultralytics import YOLO
except ImportError:
    print("Installing ultralytics...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics"])
    from ultralytics import YOLO

# Setup paths
DATASET_ROOT = '/content/drive/MyDrive/Fruits_data'
WORK_DIR = '/content/fruit_detection'
MODEL_SAVE_DIR = '/content/drive/MyDrive/fruit_detection_models'
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# Create data.yaml if not exists
yaml_path = os.path.join(WORK_DIR, 'data.yaml')
if not os.path.exists(yaml_path):
    data_yaml = {
        'path': DATASET_ROOT,
        'train': f'{DATASET_ROOT}/train/images',
        'val': f'{DATASET_ROOT}/valid/images',
        'test': f'{DATASET_ROOT}/test/images',
        'nc': 9,
        'names': ['apple', 'banana', 'cherry', 'lemon', 'orange', 'peach', 'pear', 'strawberry', 'watermelon']
    }
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)
    print(f"✓ Created data.yaml")

# Check dataset structure
print("="*70)
print("DATASET STRUCTURE CHECK")
print("="*70)
for split in ['train', 'valid', 'test']:
    split_path = os.path.join(DATASET_ROOT, split)
    if os.path.exists(split_path):
        img_count = len(os.listdir(os.path.join(split_path, 'images'))) if os.path.exists(os.path.join(split_path, 'images')) else 0
        lbl_count = len(os.listdir(os.path.join(split_path, 'labels'))) if os.path.exists(os.path.join(split_path, 'labels')) else 0
        print(f"✓ {split:6s}: {img_count:4d} images, {lbl_count:4d} labels")
    else:
        print(f"❌ {split:6s}: NOT FOUND")

# Load or create model
training_dir = os.path.join(WORK_DIR, 'fruit_detector')
last_checkpoint = os.path.join(training_dir, 'weights', 'last.pt')
resume_training = os.path.exists(last_checkpoint)

if resume_training:
    print("\n🔄 RESUME MODE: Continuing from last checkpoint")
    model = YOLO(last_checkpoint)
else:
    print("\n📚 FRESH TRAINING: Loading YOLOv8m model")
    model = YOLO('yolov8m.pt')

# Auto-detect device
device = 0 if torch.cuda.is_available() else 'cpu'
gpu_info = f"GPU: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "CPU"
print(f"Device: {gpu_info}")

print("\n" + "="*70)
print("STARTING TRAINING...")
print("="*70)

# Train model
results = model.train(
    data=yaml_path,
    epochs=1,
    imgsz=640,
    batch=16,
    patience=20,
    device=device,
    save=True,
    exist_ok=True,
    project=WORK_DIR,
    name='fruit_detector',
    resume=resume_training,
    save_period=1,
    augment=True,
    lr0=0.01,
    lrf=0.01,
    warmup_epochs=3,
)

print("\n" + "="*70)
print("✓ TRAINING COMPLETED!")
print("="*70)

# Auto-save best model to Google Drive
best_model_path = os.path.join(training_dir, 'weights', 'best.pt')
last_model_path = os.path.join(training_dir, 'weights', 'last.pt')

best_model_save_path = os.path.join(MODEL_SAVE_DIR, 'fruit_detector_best.pt')
last_model_save_path = os.path.join(MODEL_SAVE_DIR, 'fruit_detector_last.pt')

print("\nSaving models to Google Drive...")
if os.path.exists(best_model_path):
    shutil.copy(best_model_path, best_model_save_path)
    print(f"✓ Best model saved: {best_model_save_path}")

if os.path.exists(last_model_path):
    shutil.copy(last_model_path, last_model_save_path)
    print(f"✓ Last model saved: {last_model_save_path}")

# Save training config
results_save_path = os.path.join(MODEL_SAVE_DIR, 'training_results.yaml')
results_data = {
    'model': 'YOLOv8m',
    'num_classes': 9,
    'classes': ['apple', 'banana', 'cherry', 'lemon', 'orange', 'peach', 'pear', 'strawberry', 'watermelon'],
    'epochs': 100,
    'batch_size': 16,
    'image_size': 640
}
with open(results_save_path, 'w') as f:
    yaml.dump(results_data, f)
print(f"✓ Training config saved")

# Save training plots if available
for file in ['results.csv', 'confusion_matrix.png', 'results.png']:
    src = os.path.join(training_dir, file)
    if os.path.exists(src):
        dst = os.path.join(MODEL_SAVE_DIR, file)
        shutil.copy(src, dst)
        print(f"✓ {file} saved")

print(f"\n✓ All models saved to Google Drive: {MODEL_SAVE_DIR}")
print("="*70)
print("\n💡 To resume if interrupted, just re-run this cell!")
print("="*70)

ModuleNotFoundError: No module named 'ultralytics'

## 6. Evaluate Model Performance

In [ ]:
# Load best model and evaluate
best_model_path = f'{WORK_DIR}/fruit_detector/weights/best.pt'
best_model = YOLO(best_model_path)

# Evaluate on validation set
print("Evaluating on validation set...")
val_results = best_model.val()

print("\n" + "="*50)
print("VALIDATION RESULTS")
print("="*50)
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")

# Evaluate on test set
print("\n" + "="*50)
print("TEST SET EVALUATION")
print("="*50)
test_results = best_model.val(data=yaml_path, split='test')
print(f"Test mAP50: {test_results.box.map50:.4f}")
print(f"Test mAP50-95: {test_results.box.map:.4f}")

# Display class-wise metrics
print("\n" + "="*50)
print("CLASS-WISE METRICS")
print("="*50)
if hasattr(test_results, 'box'):
    for i, class_name in CLASSES.items():
        print(f"{class_name}: AP50={test_results.box.ap50[i]:.4f}, AP={test_results.box.ap[i]:.4f}")

## 7. Summary and Model Information

In [ ]:
print("="*70)
print(" FRUIT RECOGNITION MODEL - TRAINING SUMMARY")
print("="*70)

print("\n📊 DATASET INFORMATION:")
print(f"  • Framework: YOLOv8")
print(f"  • Number of Classes: 9")
print(f"  • Classes: apple, banana, cherry, lemon, orange, peach, pear, strawberry, watermelon")

print("\n📁 DATASET PATHS:")
print(f"  • Training data: {DATASET_ROOT}/train/images")
print(f"  • Validation data: {DATASET_ROOT}/valid/images")
print(f"  • Test data: {DATASET_ROOT}/test/images")

print("\n🏋️ TRAINING CONFIGURATION:")
print(f"  • Model: YOLOv8 Medium (m)")
print(f"  • Epochs: 100")
print(f"  • Batch Size: 16")
print(f"  • Image Size: 640x640")
print(f"  • Optimizer: SGD with momentum")
print(f"  • Learning Rate: 0.01 (initial)")
print(f"  • Augmentation: Enabled")
print(f"  • Save Period: Every epoch (save_period=1)")

print("\n💾 MODEL SAVE LOCATION:")
print(f"  • Google Drive: {MODEL_SAVE_DIR}")
print(f"  • Best Model: fruit_detector_best.pt")
print(f"  • Last Model: fruit_detector_last.pt")
print(f"  • Training Config: training_results.yaml")

print("\n🧪 TESTING OPTIONS AVAILABLE:")
print("  ✓ Test on images from dataset")
print("  ✓ Test on uploaded images")
print("  ✓ Test on images from Google Drive")
print("  ✓ Test on videos")
print("  ✓ Batch testing on image directories")
print("  ✓ Detailed predictions visualization")

print("\n📝 QUICK START:")
print("  # Test on Google Drive images:")
print('  test_images_from_gdrive("/content/drive/MyDrive/Fruits_data/test/images", max_images=5)')
print("\n  # Test on video:")
print("  process_video('/path/to/video.mp4')")

print("\n" + "="*70)
print(" ✅ Models auto-save to Google Drive after each epoch!")
print("="*70)